In [1]:
import json
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import itertools

In [2]:
def load_json_file(path):
    with open(path, "r") as f:
        return json.load(f)

# result = load_json_file("result.json")

In [ ]:
def cord_to_pixels(cv2_image, center_x, center_y, width, height):
    # image Height and Width
    img_height, img_width = cv2_image.shape
    # Calculate Width and Height from object
    calc_width = img_width * width
    calc_height = img_height * height
    # Calculate center X, Y
    ct_x_new = img_width * center_x
    ct_y_new = img_height * center_y
    # Calculate starting point of X and Y cordinates on image
    start_x = ct_x_new - calc_width/2
    start_y = ct_y_new - calc_height/2
    if start_x < 0:
        # start_x = calc_width/2 - ct_x_new
        start_x = start_x * -1
    if start_y < 0:
        start_y = start_y * -1
        # start_y = calc_height/2 - ct_y_new
    return start_x, start_y, calc_width, calc_height, ct_x_new, ct_y_new

def crop_image(img, start_x, start_y, width, height):
    img_copy = img.copy()
    start_x = int(start_x)
    start_y = int(start_y)
    width = int(width)
    height = int(height)

    end_x = int(start_x+width)
    end_y = int(start_y+height)

    crop = img_copy[start_y:end_y, start_x:end_x]
    return crop

In [ ]:
def main(path, colab=True):
    result = load_json_file(path)

    nucleus = {}
    onion = {}
    white = {}
    black = {}
    zebra = {}
    worm = {}

    counter = 0
    previous = None
    for object in result:
        image_name = os.path.basename(object["filename"]).split('/')[-1]
        z_cordinate = float(image_name.split("=")[-1][:-6])
        z_cordinate += counter
        counter += 1
        # if required conversion to pixels
        # z_cordinate = z_cordinate * len(result)
        image = None
        if colab:
            image = cv2.imread(object["filename"], 0)
        else:
            image = cv2.imread("images/" + image_name, 0)
        
        for obj in object["objects"]:
            name = obj["name"]
            rel_cords = obj["relative_coordinates"]
            start_x, start_y, width, height, center_x, center_y = cord_to_pixels(image, rel_cords["center_x"], rel_cords["center_y"], rel_cords["width"], rel_cords["height"])
            # Save converted coordinates
            obj["coordinates"] = {
                "start_x": start_x,
                "start_y": start_y,
                "width": width,
                "height": height,
                "center_x": center_x,
                "center_y": center_y
                }
            obj["class_name_id"] = None


            # Treshold
            cropped = crop_image(image, start_x, start_y, width, height)
            rows, cols = cropped.shape
            if "nucleus" == obj["name"]:
                ret,cropped = cv2.threshold(cropped,110,255,cv2.THRESH_BINARY_INV)
                kernel = np.ones((3, 3), np.uint8)
                cropped = cv2.morphologyEx(cropped, cv2.MORPH_OPEN, kernel, iterations=2)
                cropped = cv2.erode(cropped, kernel, iterations=4) # mozda 3 erode za nijansu bolje rezultate
            elif "white" == obj["name"]:
                cropped = cv2.adaptiveThreshold(cropped, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 141, 30)
            elif "black" == obj["name"]:
                cropped = cv2.adaptiveThreshold(cropped, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 101, 20)
            elif "zebra" == obj["name"]:
                ret,cropped = cv2.threshold(cropped,110,255,cv2.THRESH_BINARY_INV)
                # or
                # cropped = cv2.adaptiveThreshold(cropped, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 125, 1)
                # kernel = np.ones((3, 3), np.uint8)
                # cropped = cv2.morphologyEx(cropped, cv2.MORPH_ERODE, kernel, iterations=1)
            elif "onion" == obj["name"]:
                # cropped = cv2.adaptiveThreshold(cropped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 89, 25)
                # or
                cropped = cv2.adaptiveThreshold(cropped, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 119, 7)
                kernel = np.ones((3, 3), np.uint8)
                cropped = cv2.morphologyEx(cropped, cv2.MORPH_ERODE, kernel, iterations=1)
            elif "worm" == obj["name"]:
                cropped = cv2.adaptiveThreshold(cropped, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 103, 13)
                kernel = np.ones((3, 3), np.uint8)
                cropped = cv2.morphologyEx(cropped, cv2.MORPH_ERODE, kernel, iterations=1)

            # Get every white pixel
            found_treshold_values = np.where(cropped == 255)
            # X [1] - Y [0] 
            x = np.array(found_treshold_values[1]) + start_x
            y = np.array(found_treshold_values[0]) + start_y
            # zip x, y and z coordinates. itertools.repeat() is used because we have same Z coordinate per image.
            list_of_coordinates = list(zip(x, y, itertools.repeat(z_cordinate)))
            updated_coordinates = "".join(map(str, list_of_coordinates)) + "\n"
            # Saving huge amount of time compared to for loop
            updated_coordinates = updated_coordinates.replace("(", "").replace(")", "\n").replace(",", "").strip()
            # Use this to connect cordinates to avoid errors while writing. .strip() is used to remove last row and \n when appending new data.
            #    string = string + "\n" + string

            #===========================
            # Class name id is used for easy connection with objects on Z coordinate and sorting objects by type.

            # obj["confidence"] > 0.8
            if previous is None:
                class_name_id = None
                if name == "nucleus":
                    class_name_id = name + "_" + str(len(nucleus))
                    nucleus[class_name_id] = updated_coordinates
                elif name == "onion":
                    class_name_id = name + "_" + str(len(onion))
                    onion[class_name_id] = updated_coordinates
                elif name == "white":
                    class_name_id = name + "_" + str(len(white))
                    white[class_name_id] = updated_coordinates
                elif name == "black":
                    class_name_id = name + "_" + str(len(black))
                    black[class_name_id] = updated_coordinates
                elif name == "zebra":
                    class_name_id = name + "_" + str(len(zebra))
                    zebra[class_name_id] = updated_coordinates
                elif name == "worm":
                    class_name_id = name + "_" + str(len(worm))
                    worm[class_name_id] = updated_coordinates
                obj["class_name_id"] = class_name_id
            else:
                for prev in previous["objects"]:
                    if name == prev["name"]:
                        prev_x = prev["coordinates"]["center_x"]
                        prev_y = prev["coordinates"]["center_y"]
                        if name == "nucleus":
                            if (prev_x-15 <= center_x <= prev_x+15) and (prev_y-15 <= center_y <= prev_y+15):
                                class_name_id = prev["class_name_id"]
                                obj["class_name_id"] = class_name_id
                                nucleus[class_name_id] = nucleus[class_name_id] + "\n" + updated_coordinates
                        elif name == "onion":
                            if (prev_x-15 <= center_x <= prev_x+15) and (prev_y-15 <= center_y <= prev_y+15):
                                class_name_id = prev["class_name_id"]
                                obj["class_name_id"] = class_name_id
                                onion[class_name_id] = onion[class_name_id] + "\n" + updated_coordinates
                        elif name == "white":
                            if (prev_x-15 <= center_x <= prev_x+15) and (prev_y-15 <= center_y <= prev_y+15):
                                class_name_id = prev["class_name_id"]
                                obj["class_name_id"] = class_name_id
                                white[class_name_id] = white[class_name_id] + "\n" + updated_coordinates
                        elif name == "black":
                            if (prev_x-15 <= center_x <= prev_x+15) and (prev_y-15 <= center_y <= prev_y+15):
                                class_name_id = prev["class_name_id"]
                                obj["class_name_id"] = class_name_id
                                black[class_name_id] = black[class_name_id] + "\n" + updated_coordinates
                        elif name == "zebra":
                            if (prev_x-15 <= center_x <= prev_x+15) and (prev_y-15 <= center_y <= prev_y+15):
                                class_name_id = prev["class_name_id"]
                                obj["class_name_id"] = class_name_id
                                zebra[class_name_id] = zebra[class_name_id] + "\n" + updated_coordinates
                        elif name == "worm":
                            if (prev_x-15 <= center_x <= prev_x+15) and (prev_y-15 <= center_y <= prev_y+15):
                                class_name_id = prev["class_name_id"]
                                obj["class_name_id"] = class_name_id
                                worm[class_name_id] = worm[class_name_id] + "\n" + updated_coordinates

                if obj["class_name_id"] == None:
                    class_name = None
                    if name == "nucleus":
                        class_name = name + "_"+ str(len(nucleus))
                        obj["class_name_id"] = class_name
                        nucleus[class_name] = updated_coordinates
                    elif name == "onion":
                        class_name = name + "_"+ str(len(onion))
                        obj["class_name_id"] = class_name
                        onion[class_name] = updated_coordinates
                    elif name == "white":
                        class_name = name + "_"+ str(len(white))
                        obj["class_name_id"] = class_name
                        white[class_name] = updated_coordinates
                    elif name == "black":
                        class_name = name + "_"+ str(len(black))
                        obj["class_name_id"] = class_name
                        black[class_name] = updated_coordinates
                    elif name == "zebra":
                        class_name = name + "_"+ str(len(zebra))
                        obj["class_name_id"] = class_name
                        zebra[class_name] = updated_coordinates
                    elif name == "worm":
                        class_name = name + "_"+ str(len(worm))
                        obj["class_name_id"] = class_name
                        worm[class_name] = updated_coordinates
        previous = object

    return result, nucleus, onion, white, black, zebra, worm

rez, nucleus, onion, white, black, zebra, worm = main("result.json", False)

In [ ]:
# Write in files
# names = {"nucleus":nucleus, "onion":onion, "white":white, "black":black, "zebra":zebra, "worm":worm}
# for name in names:
#     if not os.path.exists("results/" + name):
#         os.makedirs("results/" + name)
# for name in names:
#     current = names.get(name)
#     for current_object in current:
#         with open("results/" + name + "/" + current_object + ".xyz", "w") as f:
#             f.writelines(current.get(current_object))

# test
import time
from multiprocessing import Process, Pool
from multiprocessing.dummy import Pool as ThreadPool
from concurrent.futures import ThreadPoolExecutor


def write_class(name, cls):
    print("started", name)
    for current_object in cls:
        with open("results/" + name + "/" + current_object + ".xyz", "w") as f:
            f.writelines(cls.get(current_object))
    return True


names = ["nucleus", "onion", "white", "black", "zebra", "worm"]

for name in names:
    if not os.path.exists("results/" + name):
        os.makedirs("results/" + name)

if __name__ == "__main__":
    data = [("nucleus", nucleus), ("onion", onion), ("white", white), ("black", black), ("zebra", zebra), ("worm", worm)]
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(write_class, name, cls) for name, cls in data]
        
        # Wait for all to complete
        for future in futures:
            future.result()



started nucleus
started onion
started white
started started zebra
blackstarted worm

